In [ ]:
# import datalad.api as dl

# hcp_path = "/Users/Shared/hcp/human-connectome-project-openaccess/"
# ds = dl.Dataset(hcp_path)

# results = ds.status(annex='all')

# ds.get("HCP1200/116726")
# ds.get("HCP1200/118528")
# ds.get("HCP1200/118528/unprocessed")



In [1]:
import os
import sys
from pathlib import Path
import datalad.api as dl
from datalad.api import Dataset
from datetime import datetime
from contextlib import contextmanager
import logging

In [2]:
# Create logs directory
os.makedirs("logs", exist_ok=True)

# Configure base logging (this will catch setup_dataset logs)
base_log_filename = f"logs/hcp_setup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(base_log_filename),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

@contextmanager
def log_get_operation(subject, data_type, log_dir="logs"):
    """
    Context manager to create separate log file for each ds.get() operation
    
    Args:
        subject: Subject ID (e.g., "116726")
        data_type: Type of data being downloaded (e.g., "T1w_MPR1", "tfMRI_MOTOR_LR")
        log_dir: Directory to save logs
    """
    
    data_type = data_type.replace('/', '_')


    # Create unique log filename
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_filename = f"{log_dir}/get_{subject}_{data_type}_{timestamp}.log"
    
    # Create file handler for this specific operation
    file_handler = logging.FileHandler(log_filename)
    file_handler.setLevel(logging.INFO)
    file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(file_formatter)
    
    # Get the root logger and add file handler
    root_logger = logging.getLogger()
    root_logger.addHandler(file_handler)
    
    # Log start of operation
    logger.info(f"Starting download: Subject {subject}, Data type {data_type}")
    logger.info(f"Operation log file: {log_filename}")
    
    try:
        yield log_filename
    except Exception as e:
        logger.error(f"Error during download: {e}")
        raise
    finally:
        logger.info(f"Completed operation for Subject {subject}, Data type {data_type}")
        # Clean up - remove the file handler
        root_logger.removeHandler(file_handler)
        file_handler.close()

In [3]:
# Configuration
REPO_URL = "https://github.com/datalad-datasets/human-connectome-project-openaccess"
LOCAL_DATASET_PATH = "/Users/zenkavi/hcp/"


In [4]:
def setup_dataset(repo_url, local_path):
    """
    Clone or update the HCP dataset
    """
    logger.info(f"Setting up dataset at {local_path}")
    
    if os.path.exists(local_path):
        logger.info("Dataset already exists, updating...")
        ds = Dataset(local_path)
        try:
            ds.update(merge=True)
        except Exception as e:
            if "dubious ownership" in str(e):
                logger.error("Git ownership issue detected!")
                logger.error("To fix this, run:")
                logger.error(f"git config --global --add safe.directory {os.path.abspath(local_path)}")
                logger.error("Or for all HCP subjects:")
                logger.error(f"find {os.path.abspath(local_path)}/HCP1200 -maxdepth 1 -type d -exec git config --global --add safe.directory {{}} \\;")
                raise
            else:
                logger.warning(f"Update failed: {e}. Continuing with existing dataset.")
    else:
        logger.info("Cloning dataset...")
        try:
            ds = dl.clone(repo_url, local_path)
        except Exception as e:
            if "dubious ownership" in str(e):
                logger.error("Git ownership issue during clone!")
                logger.error("To fix this, run:")
                logger.error(f"git config --global --add safe.directory {os.path.abspath(local_path)}")
                logger.error("Then re-run this script.")
                raise
            else:
                raise
    
    return ds

In [5]:
import shutil

def remove_unwanted_directories(ds, subject):
    """
    Remove unwanted directories using file system operations
    """
    
    keep_patterns = [
        "T1w_MPR1",
        "T1w_MPR2", 
        "T2w_SPC1",
        "tfMRI_MOTOR_LR",
        "tfMRI_MOTOR_RL",
        "tfMRI_GAMBLING_LR",
        "tfMRI_GAMBLING_RL"
    ]
    
    rm_paths = ["3T", "7T", "MEG"]

    for rm_path in rm_paths:
        logger.info(f"Removing unwanted data for subject {subject}")
        
        subject_rm_path = Path(ds.path) / "HCP1200" / subject / "unprocessed" / rm_path
        
        if subject_rm_path.exists():
            all_dirs = [d for d in subject_rm_path.iterdir() if d.is_dir()]
            
            for dir_path in all_dirs:
                dir_name = dir_path.name
                
                if dir_name not in keep_patterns:
                    logger.info(f"Removing directory: {dir_path}")
                    try:
                        shutil.rmtree(dir_path)
                        logger.info(f"Successfully removed {dir_name} for {subject}")
                    except Exception as e:
                        logger.error(f"Failed to remove {dir_name} for {subject}: {e}")
                else:
                    logger.info(f"Keeping directory: {dir_name} for {subject}")
        else:
            logger.warning(f"Subject rm directory not found: {subject_rm_path}")


In [6]:
# Your data specifications
# subject_list = ["116726", "118528"]

subject_list = ["116726", "118528", "131823", "135124", "136126", "150524", "152225", "167743", "176542"]

# Setup dataset once
# LOCAL_DATA_PATH shouldn't exist before this
ds = setup_dataset(REPO_URL, LOCAL_DATASET_PATH)

# Download each subject's data with separate logs
for subject in subject_list:
    logger.info(f"Processing subject {subject}")

    patterns = [f"HCP1200/{subject}/unprocessed/3T/T1w_MPR1/", 
                f"HCP1200/{subject}/unprocessed/3T/T1w_MPR2/",
                f"HCP1200/{subject}/unprocessed/3T/T2w_SPC1/",
                f"HCP1200/{subject}/unprocessed/3T/tfMRI_MOTOR_LR/",
                f"HCP1200/{subject}/unprocessed/3T/tfMRI_MOTOR_RL/",
                f"HCP1200/{subject}/unprocessed/3T/tfMRI_GAMBLING_LR/",
                f"HCP1200/{subject}/unprocessed/3T/tfMRI_GAMBLING_RL/"]
    
    # Download structural data
    for pattern in patterns:        
        with log_get_operation(subject, pattern) as log_file:
            try:
                result = ds.get(pattern)
                logger.info(f"Successfully downloaded {pattern}")
            except Exception as e:
                logger.error(f"Failed to download {pattern}")
        remove_unwanted_directories(ds, subject)
    

logger.info("All downloads completed")

2025-09-21 21:51:49,805 - INFO - Setting up dataset at /Users/zenkavi/hcp/
2025-09-21 21:51:49,808 - INFO - Cloning dataset...
[INFO] Attempting a clone into /Users/zenkavi/hcp 
2025-09-21 21:51:49,826 - INFO - Attempting a clone into /Users/zenkavi/hcp
[INFO] Attempting to clone from https://github.com/datalad-datasets/human-connectome-project-openaccess to /Users/zenkavi/hcp 
2025-09-21 21:51:49,827 - INFO - Attempting to clone from https://github.com/datalad-datasets/human-connectome-project-openaccess to /Users/zenkavi/hcp
[INFO] Start enumerating objects 
2025-09-21 21:51:50,454 - INFO - Start enumerating objects
2025-09-21 21:51:50,457 - INFO - Finished enumerating objects
[INFO] Start counting objects 
2025-09-21 21:51:50,458 - INFO - Start counting objects
2025-09-21 21:51:50,459 - INFO - remote: Counting objects:   1% (1/90)
2025-09-21 21:51:50,460 - INFO - remote: Counting objects:   2% (2/90)
2025-09-21 21:51:50,460 - INFO - remote: Counting objects:   3% (3/90)
2025-09-21 2

install(ok): /Users/zenkavi/hcp (dataset)


[INFO] Completed clone attempts for Dataset(/Users/zenkavi/hcp/HCP1200/116726) 
2025-09-21 21:51:52,173 - INFO - Completed clone attempts for Dataset(/Users/zenkavi/hcp/HCP1200/116726)
2025-09-21 21:51:53,846 - INFO - Clear progress bars
2025-09-21 21:51:53,847 - INFO - Refresh progress bars
[INFO] Attempting a clone into /Users/zenkavi/hcp/HCP1200/116726/unprocessed 
2025-09-21 21:51:53,986 - INFO - Attempting a clone into /Users/zenkavi/hcp/HCP1200/116726/unprocessed
[INFO] Attempting to clone from http://store.datalad.org/98f/66652-2b46-11ea-8752-0025904abcb0 to /Users/zenkavi/hcp/HCP1200/116726/unprocessed 
2025-09-21 21:51:53,987 - INFO - Attempting to clone from http://store.datalad.org/98f/66652-2b46-11ea-8752-0025904abcb0 to /Users/zenkavi/hcp/HCP1200/116726/unprocessed


install(ok): /Users/zenkavi/hcp/HCP1200/116726 (dataset) [Installed subdataset in order to get /Users/zenkavi/hcp/HCP1200/116726/unprocessed/3T/T1w_MPR1]


[INFO] Completed clone attempts for Dataset(/Users/zenkavi/hcp/HCP1200/116726/unprocessed) 
2025-09-21 21:51:55,435 - INFO - Completed clone attempts for Dataset(/Users/zenkavi/hcp/HCP1200/116726/unprocessed)
2025-09-21 21:51:56,870 - INFO - Clear progress bars
2025-09-21 21:51:56,870 - INFO - Refresh progress bars
2025-09-21 21:51:57,003 - INFO - Start annex operation


...

get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/176542_3T_tfMRI_MOTOR_RL.nii.gz (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/176542_3T_tfMRI_MOTOR_RL_SBRef.nii.gz (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/LINKED_DATA/EPRIME/176542_3T_MOTOR_run1_TAB.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/LINKED_DATA/EPRIME/EVs/Sync.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/LINKED_DATA/EPRIME/EVs/cue.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/LINKED_DATA/EPRIME/EVs/lf.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/LINKED_DATA/EPRIME/EVs/lh.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/LINKED_DATA/EPRIME/EVs/rf.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_MOTOR_RL/LINKED_DATA/EPRIME/EVs/rh.txt (file) [from 

2025-09-21 22:15:47,791 - INFO - Start annex action: {'command': 'get', 'file': '3T/tfMRI_GAMBLING_LR/176542_3T_tfMRI_GAMBLING_LR.nii.gz', 'input': ['3T/tfMRI_GAMBLING_LR'], 'key': 'MD5E-s284906586--ebb6ef0950b007973dc5eaf41b868390.nii.gz', 'note': 'from datalad...'}
2025-09-21 22:15:47,792 - INFO - 0.09%
2025-09-21 22:15:47,898 - INFO - 0.37%
2025-09-21 22:15:48,059 - INFO - 1.47%
2025-09-21 22:15:48,196 - INFO - 2.39%
2025-09-21 22:15:48,300 - INFO - 4.51%
2025-09-21 22:15:48,401 - INFO - 6.35%
2025-09-21 22:15:48,579 - INFO - 7.45%
2025-09-21 22:15:48,689 - INFO - 8.74%
2025-09-21 22:15:48,804 - INFO - 10.03%
2025-09-21 22:15:48,909 - INFO - 11.78%
2025-09-21 22:15:49,106 - INFO - 13.53%
2025-09-21 22:15:49,207 - INFO - 15%
2025-09-21 22:15:49,324 - INFO - 16.01%
2025-09-21 22:15:49,426 - INFO - 17.67%
2025-09-21 22:15:49,627 - INFO - 18.31%
2025-09-21 22:15:49,786 - INFO - 19.14%
2025-09-21 22:15:49,948 - INFO - 19.69%
2025-09-21 22:15:50,059 - INFO - 20.61%
2025-09-21 22:15:50,165

get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/176542_3T_tfMRI_GAMBLING_LR.nii.gz (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/176542_3T_tfMRI_GAMBLING_LR_SBRef.nii.gz (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/LINKED_DATA/EPRIME/176542_3T_GAMBLING_run2_TAB.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/LINKED_DATA/EPRIME/EVs/GAMBLING_Stats.csv (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/LINKED_DATA/EPRIME/EVs/Sync.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/LINKED_DATA/EPRIME/EVs/loss.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/LINKED_DATA/EPRIME/EVs/loss_event.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_LR/LINKED_DATA/EPRIME/EVs/neut_event.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed

2025-09-21 22:16:22,137 - INFO - Start annex action: {'command': 'get', 'file': '3T/tfMRI_GAMBLING_RL/176542_3T_tfMRI_GAMBLING_RL.nii.gz', 'input': ['3T/tfMRI_GAMBLING_RL'], 'key': 'MD5E-s357674455--ad7b3204975841b8ea48e78947950574.nii.gz', 'note': 'from datalad...'}
2025-09-21 22:16:22,138 - INFO - 0.07%
2025-09-21 22:16:22,276 - INFO - 0.22%
2025-09-21 22:16:22,406 - INFO - 0.51%
2025-09-21 22:16:22,507 - INFO - 1.25%
2025-09-21 22:16:22,670 - INFO - 1.61%
2025-09-21 22:16:22,796 - INFO - 2.05%
2025-09-21 22:16:22,938 - INFO - 2.49%
2025-09-21 22:16:23,084 - INFO - 3.22%
2025-09-21 22:16:23,186 - INFO - 3.44%
2025-09-21 22:16:23,322 - INFO - 3.74%
2025-09-21 22:16:23,464 - INFO - 4.1%
2025-09-21 22:16:23,600 - INFO - 4.62%
2025-09-21 22:16:23,708 - INFO - 4.84%
2025-09-21 22:16:23,862 - INFO - 5.42%
2025-09-21 22:16:23,964 - INFO - 5.64%
2025-09-21 22:16:24,099 - INFO - 6.16%
2025-09-21 22:16:24,231 - INFO - 6.3%
2025-09-21 22:16:24,359 - INFO - 6.67%
2025-09-21 22:16:24,459 - INFO -

get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/176542_3T_tfMRI_GAMBLING_RL.nii.gz (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/176542_3T_tfMRI_GAMBLING_RL_SBRef.nii.gz (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/LINKED_DATA/EPRIME/176542_3T_GAMBLING_run1_TAB.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/LINKED_DATA/EPRIME/EVs/GAMBLING_Stats.csv (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/LINKED_DATA/EPRIME/EVs/Sync.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/LINKED_DATA/EPRIME/EVs/loss.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/LINKED_DATA/EPRIME/EVs/loss_event.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed/3T/tfMRI_GAMBLING_RL/LINKED_DATA/EPRIME/EVs/neut_event.txt (file) [from datalad...]
get(ok): HCP1200/176542/unprocessed

In [ ]:
# SIMPLEST CASE

# import datalad.api as dl

# REPO_URL = "https://github.com/datalad-datasets/human-connectome-project-openaccess"
# LOCAL_DATASET_PATH = "/Users/zenkavi/hcp/"

# ds = dl.clone(REPO_URL, LOCAL_DATASET_PATH)

# ds.get("HCP1200/116726/unprocessed/3T/T1w_MPR1/")
# ds.get("HCP1200/116726/unprocessed/3T/T1w_MPR2/")
# ds.get("HCP1200/116726/unprocessed/3T/T2w_SPC1/")
# ds.get("HCP1200/116726/unprocessed/3T/tfMRI_MOTOR_LR/")
# ds.get("HCP1200/116726/unprocessed/3T/tfMRI_MOTOR_RL/")
# ds.get("HCP1200/116726/unprocessed/3T/tfMRI_GAMBLING_LR/")
# ds.get("HCP1200/116726/unprocessed/3T/tfMRI_GAMBLING_RL/")

# remove_unwanted_directories(ds, "116726")